## Bài 1

In [1]:
import sqlite3
import pandas as pd
import math

df = pd.DataFrame({
    'A': [10, 20, 30, 40, 50],
    'B': [12, 24, 33, 47, 52]
})

# Tạo kết nối SQLite in-memory
conn = sqlite3.connect(":memory:")
df.to_sql("my_table", conn, index=False, if_exists="replace")

# Truy vấn các giá trị cần tính với CTE
query = """
WITH Data AS (
    SELECT
        COUNT(*) as n,
        SUM(A) as sum_a,
        SUM(B) as sum_b,
        SUM(A * B) as sum_ab,
        SUM(A * A) as sum_a2,
        SUM(B * B) as sum_b2
    FROM my_table
)
SELECT n, sum_a, sum_b, sum_ab, sum_a2, sum_b2 FROM Data;
"""

row = conn.execute(query).fetchone()
n, sum_a, sum_b, sum_ab, sum_a2, sum_b2 = row

# Áp dụng công thức Pearson
numerator = n * sum_ab - sum_a * sum_b
denominator = math.sqrt(n * sum_a2 - sum_a**2) * math.sqrt(n * sum_b2 - sum_b**2)
r = numerator / denominator

print(f"Hệ số tương quan Pearson r = {r}")

Hệ số tương quan Pearson r = 0.9924052482502039


## Bài 2


In [2]:
import sqlite3
import pandas as pd

# Tạo dữ liệu dạng long format bằng dictionary
data_dict = {
    'Day': ['Day 1']*3 + ['Day 2']*3 + ['Day 3']*3 + ['Day 4']*3,
    'Model': ['A', 'B', 'C'] * 4,
    'Score': [8, 9, 7, 7.5, 8.5, 7, 6, 7, 8, 7, 6, 5]
}
data = pd.DataFrame(data_dict)

# Kết nối SQLite in-memory
conn = sqlite3.connect(':memory:')

# Đẩy dữ liệu vào SQLite
data.to_sql("test_scores", conn, index=False, if_exists="replace")

# Kiểm tra xem có dữ liệu chưa
print(pd.read_sql("SELECT * FROM test_scores LIMIT 5;", conn))

# Trung bình theo mẫu
query_model_avg = """
SELECT Model, COUNT(*) AS n, AVG(Score) AS avg_score
FROM test_scores
GROUP BY Model;
"""

# Trung bình theo ngày
query_day_avg = """
SELECT Day, COUNT(*) AS n, AVG(Score) AS avg_score
FROM test_scores
GROUP BY Day;
"""

# Trung bình toàn bộ
query_overall_avg = """
SELECT AVG(Score) AS overall_avg FROM test_scores;
"""

print("\n-- Trung bình theo mẫu --")
print(pd.read_sql(query_model_avg, conn))

print("\n-- Trung bình theo ngày --")
print(pd.read_sql(query_day_avg, conn))

print("\n-- Trung bình toàn bộ --")
print(pd.read_sql(query_overall_avg, conn))

     Day Model  Score
0  Day 1     A    8.0
1  Day 1     B    9.0
2  Day 1     C    7.0
3  Day 2     A    7.5
4  Day 2     B    8.5

-- Trung bình theo mẫu --
  Model  n  avg_score
0     A  4      7.125
1     B  4      7.625
2     C  4      6.750

-- Trung bình theo ngày --
     Day  n  avg_score
0  Day 1  3   8.000000
1  Day 2  3   7.666667
2  Day 3  3   7.000000
3  Day 4  3   6.000000

-- Trung bình toàn bộ --
   overall_avg
0     7.166667


In [ ]:
# Trung bình theo mẫu
query_model_avg = """
SELECT Model, COUNT(*) AS n, AVG(Score) AS avg_score
FROM test_scores
GROUP BY Model;
"""

# Trung bình theo ngày
query_day_avg = """
SELECT Day, COUNT(*) AS n, AVG(Score) AS avg_score
FROM test_scores
GROUP BY Day;
"""

# Trung bình toàn bộ
query_overall_avg = """
SELECT AVG(Score) as overall_avg FROM test_scores;
"""

print("\n-- Trung bình theo mẫu --")
print(pd.read_sql(query_model_avg, conn))

print("\n-- Trung bình theo ngày --")
print(pd.read_sql(query_day_avg, conn))

print("\n-- Trung bình toàn bộ --")
print(pd.read_sql(query_overall_avg, conn))



-- Trung bình theo mẫu --
  Model  n  avg_score
0     A  4      7.125
1     B  4      7.625
2     C  4      6.750

-- Trung bình theo ngày --
     Day  n  avg_score
0  Day 1  3   8.000000
1  Day 2  3   7.666667
2  Day 3  3   7.000000
3  Day 4  3   6.000000

-- Trung bình toàn bộ --
   overall_avg
0     7.166667


## Bài 3

In [3]:
import sqlite3
import pandas as pd

# Tạo DB và dữ liệu mẫu
conn = sqlite3.connect(':memory:')
conn.execute("CREATE TABLE flights (departure_time INTEGER)")
departure_times = [0, 5, 30, 830, 945, 1445, 2359]
conn.executemany("INSERT INTO flights (departure_time) VALUES (?)", [(t,) for t in departure_times])

# Truy vấn format lại giờ
query = """
SELECT
  departure_time,
  printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
FROM flights;
"""

df = pd.read_sql(query, conn)
print(df)

   departure_time formatted_time
0               0          00:00
1               5          00:05
2              30          00:30
3             830          08:30
4             945          09:45
5            1445          14:45
6            2359          23:59


## Bài 4


In [4]:
import sqlite3
import pandas as pd
import numpy as np

# Giả sử có bảng giá trị
values = [10, 12, 11, 13, 10, 9, 12, 100]
data = pd.DataFrame({'value': values})  # 100 là ngoại lệ

# Tạo DB SQLite in-memory và nạp dữ liệu
conn = sqlite3.connect(':memory:')
data.to_sql('my_table', conn, index=False, if_exists='replace')

# Lấy dữ liệu từ SQL để tính median & MAD trong Python
df = pd.read_sql("SELECT value FROM my_table", conn)

# Bước 1: Tính median
median = df['value'].median()

# Bước 2: Tính MAD
mad = (df['value'] - median).abs().median()

# Bước 3: Tìm giá trị ngoại lệ (dùng lại SQLite để lọc)
threshold = 1.5 * mad
query = f"""
SELECT *
FROM my_table
WHERE ABS(value - {median}) > {threshold};
"""
outliers = pd.read_sql(query, conn)
print("-- Ngoại lệ được phát hiện --")
print(outliers)

-- Ngoại lệ được phát hiện --
   value
0      9
1    100


## Bài 5


In [7]:
import sqlite3
import pandas as pd

# Kết nối SQLite (trong bộ nhớ)
conn = sqlite3.connect(':memory:')

# Tạo bảng Patient
conn.execute('''
CREATE TABLE Patient (
    last_name TEXT,
    weight REAL,
    height REAL
)
''')

# Thêm dữ liệu mẫu bằng list of dictionaries và executemany
patients_data = [
    {'last_name': 'Nguyen', 'weight': 65, 'height': 170},
    {'last_name': 'Nguyen', 'weight': 65.5, 'height': 171},
    {'last_name': 'Tran', 'weight': 60, 'height': 168},
    {'last_name': 'Nguyen', 'weight': 70, 'height': 172},
    {'last_name': 'Tran', 'weight': 59.9, 'height': 168},
    {'last_name': 'Le', 'weight': 50, 'height': 160}
]

patients_tuple = [(p['last_name'], p['weight'], p['height']) for p in patients_data]
conn.executemany('INSERT INTO Patient VALUES (?, ?, ?)', patients_tuple)

# Truy vấn tìm các cặp có thể là cùng một người
query = '''
SELECT
    A.last_name, A.weight AS weight1, B.weight AS weight2,
    A.height AS height1, B.height AS height2,
    ABS(A.weight - B.weight) AS weight_diff,
    CASE
        WHEN A.last_name = B.last_name AND ABS(A.weight - B.weight) <= 1 THEN 1
        ELSE 0
    END AS is_same_person
FROM Patient A
JOIN Patient B
    ON A.rowid < B.rowid
'''

# Hiển thị kết quả
df = pd.read_sql(query, conn)
print(df.loc[(df['is_same_person'] == 1)])

   last_name  weight1  weight2  height1  height2  weight_diff  is_same_person
0     Nguyen     65.0     65.5    170.0    171.0          0.5               1
10      Tran     60.0     59.9    168.0    168.0          0.1               1
